In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd

import pydicom as dicom
import os
import cv2
import PIL # optional
import tensorflow as tf
import shutil

In [ ]:
import os
kf = pd.DataFrame(columns=['data_set_type','StudyInstanceUID','Sub_folder','image_file_name','image_file_path'])
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        kf.loc[kf.shape[0]] = [dirname.split('/')[-3], dirname.split('/')[-2],dirname.split('/')[-1],filename,os.path.join(dirname, filename)]

df = pd.DataFrame(kf.drop([0,1,2],axis=0).values,columns=kf.columns)
df

In [ ]:
import os
import SimpleITK
import cv2
from tqdm import tqdm
import shutil
 
def convert_from_dicom_to_jpg(dcm_image_path, output_jpg_path):
           
    ds_array = SimpleITK.ReadImage(dcm_image_path)  
    img_array = SimpleITK.GetArrayFromImage(ds_array)  
            
    shape = img_array.shape
    img_array = np.reshape(img_array, (shape[1], shape[2]))  
    high = np.max(img_array)
    low = np.min(img_array)
        
    lungwin = np.array([low*1.,high*1.])
    newimg = (img_array-lungwin[0])/(lungwin[1]-lungwin[0])
    newimg = (newimg*255).astype('uint8')
    stacked_img = np.stack((newimg,) * 3, axis=-1)
    x_imag = tf.keras.preprocessing.image.array_to_img(stacked_img)
    
    # required image format and size as a input to Deep learning network
    
    ht = tf.keras.preprocessing.image.img_to_array(x_imag.resize([256,256]))
    
    cv2.imwrite(output_jpg_path, ht, [int(cv2.IMWRITE_JPEG_QUALITY), 10])
    #return ht

In [ ]:
# Use this file only if the images are readily avaiable in png format 
# We have converted dicom to png and upoaded to "/Kaggle/input/" folder
itr = 0

if os.path.exists('/kaggle/working/siim-covid19-detection/train/') == True:
    shutil.rmtree('/kaggle/working/')
if os.path.exists('/kaggle/working/siim-covid19-detection/test/') == True:
    shutil.rmtree('/kaggle/working/')
        
os.makedirs('/kaggle/working/siim-covid19-detection/test/')
os.makedirs('/kaggle/working/siim-covid19-detection/train/')

for i in df['data_set_type'].values:
    if i == 'train':
        dcm_image_path = df['image_file_path'].values[itr]
        output_jpg_path = '/kaggle/working/siim-covid19-detection/train/' + df['StudyInstanceUID'].values[itr] +'_'+ df['image_file_name'].values[itr].split('.')[0] + '.png'
        convert_from_dicom_to_jpg(dcm_image_path, output_jpg_path)
    else:
        dcm_image_path = df['image_file_path'].values[itr]
        output_jpg_path = '/kaggle/working/siim-covid19-detection/test/' + df['StudyInstanceUID'].values[itr] +'_'+ df['image_file_name'].values[itr].split('.')[0] + '.png'
        convert_from_dicom_to_jpg(dcm_image_path, output_jpg_path)
        
    itr = itr + 1
    print(itr)